In [1]:
import pandas as pd
import numpy as np

In [64]:
file = ('Final.csv')

In [96]:
df = pd.DataFrame(pd.read_csv(file))

In [198]:
# df.head()#[['full_text','id','location','user']]

# new columns

In [196]:
# clean extra characters and split hashtags
h = df['hashtag']
h = h.str.replace("']'","")
# h.iloc[10][-1]
h_df = h.str.split(expand=True).apply(lambda x: x.str.replace('\W', ''))
# name columns
cols =['hashtag_0' + str(ix+1) if len(str(ix+1)) < 2 else 'hashtag_'+ str(ix+1) for ix in range(0, len(h_df.columns)) ]
for ix in h_df.columns:
    h_df.rename(columns={ix:cols[ix]},inplace=True)

In [195]:
# retweet_hashtags
rh = df['retweet_hashtags']
rh_df = rh.str.split(',', expand=True).apply(lambda x: x.str.replace('\W', ''))
# name columns
cols =['rt_hashtag_0' + str(ix+1) if len(str(ix+1)) < 2 else 'rt_hashtag_'+ str(ix+1) for ix in range(0, len(rh_df.columns)) ]
for ix in rh_df.columns:
    rh_df.rename(columns={ix:cols[ix]},inplace=True)

In [194]:
# retweet_mentions 
rm = df['retweet_mentions']
rm_df = rm.str.split(',', expand=True).apply(lambda x: x.str.replace('\W', ''))
# name columns
cols =['rt_mentions_0' + str(ix+1) if len(str(ix+1)) < 2 else 'rt_mentions_'+ str(ix+1) for ix in range(0, len(rm_df.columns)) ]
for ix in rm_df.columns:
    rm_df.rename(columns={ix:cols[ix]},inplace=True)

In [193]:
# user_mentions
um = df['user_mentions']
um_df = um.str.split(',', expand=True).apply(lambda x: x.str.replace('\W', ''))
# name columns
cols =['user_mentions_0' + str(ix+1) if len(str(ix+1)) < 2 else 'user_mentions_'+ str(ix+1) for ix in range(0, len(um_df.columns)) ]

for ix in um_df.columns:
    um_df.rename(columns={ix:cols[ix]},inplace=True)

# Create list of hashtags

In [174]:
# create list of all hashtags
hashtag_list = []
for col in h_df.columns:
    vals = h_df[col].values
    hashtag_list.extend(vals)

hastag_list_remove_space = [val for val in hashtag_list if val != '' ]
uni_hashtags = list(set(hastag_list_remove_space))
uni_hashtags.remove(None)

In [248]:
from collections import Counter
hash_list_clean = [val for val in hastag_list_remove_space if val != None ]
hashtag_counts = Counter(hash_list_clean)
hash_dict = dict(hashtag_counts)
hashtag_counts_df = pd.DataFrame.from_dict(hash_dict, orient='index', columns=['counts']).reset_index().sort_values('counts', ascending=False)

In [250]:
hashtag_counts_df.to_csv('hashtag_counts.csv')

In [180]:
final_df = pd.DataFrame(uni_hashtags, columns=['hashtags'])

In [181]:
final_df.to_csv('hashtags.csv')

# Merge new columns with df

In [300]:
merge_df = pd.concat([df, h_df, rh_df, rm_df, um_df], axis=1)

In [301]:
merge_df['user'] = merge_df['user'].apply(lambda x: x.replace('\W', ''))

# Finding the 'why'

In [302]:
org_list = ['QIMRBerghofer','GarvanInstitute',
            'WEHI_research','MCRI_for_kids',
            'BakerResearchAu','blackdoginst',
            'CentenaryInst','jeansforgenesau',
            'TheFlorey','sahmriAU',
            'WestmeadInst','PeterMacCC',
            'EyeResearchAus','unimelb',
            'ANUmedia','UNSW',
            'MonashUni','Sydney_Uni',
            'uwanews','UniversitySA',
            'UQ_News','UniofAdelaide',
            'UTSEngage','UTAS_',
            'Flinders','Uni_Newcastle',
            'UOW','QUT',
            'latrobe','Macquarie_Uni']
keyword_list = ['Government','Policy','Funding','NHRMC','Research','Medical','Health','Grant',
                'National Medical Health Research Council']

In [342]:
# Not retweets
tweet_df = merge_df[~merge_df['full_text'].str.contains('RT')]

def find_ord(text):
    tmp_list=[]
    for ix in range(0, len(org_list)):
        if org_list[ix].lower() in text.lower():
            tmp_list.append(org_list[ix])
    return(tmp_list)

def find_key_word(text):
    tmp_list= []
    for ix in range(0, len(keyword_list)):
        if keyword_list[ix].lower() in text.lower():
            tmp_list.append(org_list[ix])
    return(tmp_list)

merge_df['linked_organisation'] = merge_df['full_text'].map(lambda x : find_ord(x) )
merge_df['linked_keyword'] = merge_df['full_text'].map(lambda x : find_key_word(x) )


In [406]:
h_df

,hashtag_01,hashtag_02,hashtag_03,hashtag_04,hashtag_05,hashtag_06,hashtag_07,hashtag_08,hashtag_09,hashtag_10,hashtag_11,hashtag_12,hashtag_13,hashtag_14,hashtag_15,hashtag_16,hashtag_17
0,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [405]:
# what hashtags link up to what key words
merge_df.to_csv('merge_df.csv')

# New heading

In [199]:
merge_df

,full_text,id,location,user,hashtag_01,hashtag_02,hashtag_03,hashtag_04,hashtag_05,hashtag_06,...,user_mentions_42,user_mentions_43,user_mentions_44,user_mentions_45,user_mentions_46,user_mentions_47,user_mentions_48,user_mentions_49,user_mentions_50,user_mentions_51
0,RT @CCNSWResearch: Congrats to our research gr...,1030936289846026240,"Sydney, New South Wales",Michelle McDonald,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,RT @CCNSWResearch: Congrats to our research gr...,1030412713120288769,"Newcastle, New South Wales",DrGillianGould,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,RT @CCNSWResearch: Congrats to our research gr...,1030408764971569152,"Turrbal & Jagera country, Qld",abbey diaz,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,RT @CCNSWResearch: Congrats to our research gr...,1030371516574097409,Australia,Women in STEMM Aust,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,RT @CCNSWResearch: Congrats to our research gr...,1030360259376832512,South Australia,Sarah Boyle 👩🏻‍🔬🎀,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,RT @CCNSWResearch: Congrats to our research gr...,1030356215937683456,Southern hemisphere,gail simmons,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,RT @CCNSWResearch: Congrats to our research gr...,1030356202830454785,NaN,Anna DeFazio-Poidevin,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,RT @CCNSWResearch: Congrats to our research gr...,1030356181460480000,"Sydney, Australia",Garvan Institute,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,RT @CCNSWResearch: Congrats to our research gr...,1030330515600039937,"Darwin, Northern Territory",Menzies Research,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,RT @CCNSWResearch: Congrats to our research gr...,1030295727753158657,"Sydney, Australia",Franklin Women,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
